In [1]:
%cd  G:\Shared drives\INVent\mfk58\Dataset\

#ensure file directory is correct

%ls 

G:\Shared drives\INVent\mfk58\Dataset
 Volume in drive G is Google Drive
 Volume Serial Number is 1983-1116

 Directory of G:\Shared drives\INVent\mfk58\Dataset

05/30/2023  03:50 PM    <DIR>          .
05/26/2023  01:31 PM    <DIR>          ..
05/30/2023  03:50 PM    <DIR>          CCF_Pre_Resampled
05/30/2023  03:53 PM    <DIR>          UH_Pre_Resampled
               0 File(s)              0 bytes
               4 Dir(s)  322,071,490,560 bytes free


In [2]:
%pip install Pillow
%pip install numpy
import glob
#import numpy

import SimpleITK as sitk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [43]:


#check some basic sanity check information to see if the masks are the same
def hasMask(name):
    files = glob.glob(name+"/*")
    if(len(files)<2):
        return False

    """ 
    f1 = sitk.ReadImage(files[0], imageIO="MetaImageIO")
    f2 = sitk.ReadImage(files[1], imageIO="MetaImageIO")

    print("running " +name)
    if not (f1.GetSize()== f2.GetSize()):
        return False
    """

    return True


def axial(name):
    files = glob.glob(name+"/*")
    if("ax" in files[0] and "ax" in files[1]):
        return True
    return False



CCF_count = 0

CCF_missing = [] #missing masks
CCF_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled/*'):
    
    if not (hasMask(name)):
        CCF_missing.append(name)
    if axial(name):
        CCF_count +=1
        CCF_list.append(name)
    

print("Total CCF: " + str(CCF_count))
print("Missing masks for CCF:" + str(CCF_missing))




UH_count = 0

UH_missing = [] #missing masks
UH_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled/*'):
    

    if not (hasMask(name)):
        UH_missing.append(name)
    if axial(name):
        UH_count +=1
        UH_list.append(name)

    

print("Total UH: " + str(UH_count))
print("Missing masks for UH:" + str(UH_missing))

print("Total: " +str(UH_count+CCF_count))
print("Expected: 165")


Total CCF: 66
Missing masks for CCF:[]
Total UH: 99
Missing masks for UH:[]
Total: 165
Expected: 165


Calculating the greater statistics

In [15]:
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [81]:
import pandas as pd
import numpy as np

split = pd.read_excel(r'G:\Shared drives\INVent\mfk58\Dataset-split.xlsx')
split = split.to_numpy();
#print(split[[1,2]])



names = UH_list
adjusted = []
for end in names:
    end = "UH_" + end.rsplit("\\",1)[-1] + "_pre_ax_raw_resampled"
    adjusted.append(end)

names = CCF_list
for end in names:
    end = "CCF_" + end.rsplit("\\",1)[-1] + "_pre_ax_raw_resampled"
    adjusted.append(end)


#these files do not match the general naming scheme, but they are present
adjusted = list(map(lambda x: x.replace("CCF_RectalCA_011_pre_ax_raw_resampled", "CCF_RectalCA_011_pre_ax_resampled"), adjusted))
adjusted = list(map(lambda x: x.replace("CCF_RectalCA_060_pre_ax_raw_resampled", "CCF_RectalCA_060_pre_ax_resampled"), adjusted))

print("Files that are not in the splitting list")
for name in adjusted:
    if not(np.isin(name,split)):
        print(name)


Files that are not in the splitting list


In [79]:

x = split[:,[0,2]].flatten()
x = np.append(x, split[0:10,3])
x.sort()
#print(x)

count = 0
for name in x:

    if name.split('_',1)[0] == "UH":
        break
    count += 1
    
print("Validation set CCF: " + str(count))
print("Validation set UH: " + str(len(x)-count))

print("Training set CCF: " + str(CCF_count-count))
print("Training set UH: " + str(UH_count-len(x)+count))




Validation set CCF: 47
Validation set UH: 67
Training set CCF: 19
Training set UH: 32
